### Importing Libs

In [1]:
import pandas as pd
from datetime import datetime

### Donations dataset
Set consisting of three CSVs:
- donations;
- year of membership;
- emails read;

---> analyzing whether the member status is annual or just the most recent;

---> analyzing the email table to understand the timestamp of the 'week' column;

#### Reading the datasets

In [2]:
year_joined = pd.read_csv("/Users/dellacorte/py-projects/data-science/time-series-pocket-reference/getting-time-series-datasets/datasets/year_joined.csv")

donations = pd.read_csv("/Users/dellacorte/py-projects/data-science/time-series-pocket-reference/getting-time-series-datasets/datasets/donations.csv")

emails = pd.read_csv("/Users/dellacorte/py-projects/data-science/time-series-pocket-reference/getting-time-series-datasets/datasets/emails.csv")

#### Exploratory Data Analysis

In [3]:
year_joined = pd.read_csv("/Users/dellacorte/py-projects/data-science/time-series-pocket-reference/getting-time-series-datasets/datasets/year_joined.csv")

# analyzing whether the member status is annual or just the most recent
year_joined.groupby("user").count().groupby("userStats").count()

,yearJoined
userStats,
1,1000


Checking the thousand records, they only have one status, so the year they joined will probably be yearJoined, accompanied by a status that could be the current status or when they joined.

In [4]:
# analyzing the email table to understand the timestamp of the 'week' column
emails = pd.read_csv("/Users/dellacorte/py-projects/data-science/time-series-pocket-reference/getting-time-series-datasets/datasets/emails.csv")
# emails.head()
emails.dtypes

#empty_emails = emails[emails.emailsOpened < 1]
#empty_emails

emailsOpened    float64
user            float64
week             object
dtype: object

There is a possibility that null weeks are not depicted in the table or members always have at least one email event. It's difficult to accept the possibility of always having an email event, so to do this, we can analyze the history of just one user:

In [5]:
user_998 = emails[emails.user == 998]
user_998

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


We can notice that some weeks are missing. For example, there are no email events after December 18, 2017. We can go further and check mathematically:

In [6]:
# converting object to datetime
emails['week'] = pd.to_datetime(emails['week'])

# member's membership time
user_membership = (max(emails[emails.user == 998].week) - 
                      min(emails[emails.user == 998].week)).days/7

user_membership

25.0

In [7]:
# number of corresponding weeks of data for user = 998
quantity_weeks_data_998 = emails[emails.user == 998].shape
quantity_weeks_data_998

(24, 3)

We have 24 rows, but we should have 26. This shows that a few weeks of this user's data is missing. By the way, we could also run this calculation for all users simultaneously.

Filling in all missing weeks for all users of the dataset

In [8]:
complete_idx = pd.MultiIndex.from_product((set(emails.week),
                                          set(emails.user)))

We will use this index to re-index the original table and fill in the missing values - in this case with 0, assuming that if nothing is recorded it means there was nothing to record.
We'll also redefine the index to make user and week information available as columns, and then name those columns:

In [9]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value = 0).reset_index()
        
all_email.columns = ['week', 'user', 'emailsOpened']

In [10]:
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
50665,2015-02-09,998.0,0.0
67374,2015-02-16,998.0,0.0
85161,2015-02-23,998.0,0.0
90551,2015-03-02,998.0,0.0
2694,2015-03-09,998.0,0.0
...,...,...,...
60906,2018-04-30,998.0,3.0
63601,2018-05-07,998.0,3.0
45275,2018-05-14,998.0,3.0
22637,2018-05-21,998.0,3.0


As we do not have the exact date on which a member started receiving emails, we are unable to implement an objective *cutoff*. With this, we will apply the rule that, for each user, we will determine the *start_date* and *end_date* cutoff points by grouping the Email DataFrame by user and selecting the maximum and minimum values ​​for the week:

In [11]:
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()

cutoff_dates = cutoff_dates.reset_index()

cutoff_dates

,index,user,min,max
0,0,1.0,2015-06-29,2018-05-28
1,1,3.0,2018-03-05,2018-04-23
2,2,5.0,2017-06-05,2018-05-28
3,3,6.0,2016-12-05,2018-05-28
4,4,9.0,2016-07-18,2018-05-28
...,...,...,...,...
534,534,991.0,2016-10-24,2016-10-24
535,535,992.0,2015-02-09,2015-07-06
536,536,993.0,2017-09-11,2018-05-28
537,537,995.0,2016-09-05,2018-05-28


We will drop the lines from the DataFrame that do not contribute much to the chronology:

In [12]:
for _, row in cutoff_dates.iterrows():
    user       = row['user']
    start_date = row['min']
    end_date   = row['max']

all_email.drop(all_email[all_email.user == user]
    [all_email.week < start_date].index, inplace = True)

all_email.drop(all_email[all_email.user == user]
    [all_email.week > end_date].index, inplace = True)

all_email

/var/folders/vb/yxjjx1m94n93ty4t50m8f2b40000gn/T/ipykernel_28600/1576256864.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == user]
/var/folders/vb/yxjjx1m94n93ty4t50m8f2b40000gn/T/ipykernel_28600/1576256864.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == user]


,week,user,emailsOpened
0,2017-02-06,1.0,3.0
1,2017-02-06,3.0,0.0
2,2017-02-06,5.0,0.0
3,2017-02-06,6.0,0.0
4,2017-02-06,9.0,3.0
...,...,...,...
93241,2017-08-21,987.0,3.0
93242,2017-08-21,991.0,0.0
93243,2017-08-21,992.0,0.0
93244,2017-08-21,993.0,0.0


#### Relating email and donation data

We can *downsample* the donation data to turn it into a weekly time series that is comparable to email data. We will aggregate the *timestamps* into weekly periods when summing them.

In [13]:
# converting a string character to a data class
donations.timestamp = pd.to_datetime(donations.timestamp)

# setting the timestamp as an index
donations.set_index('timestamp', inplace = True)

# grouping and adding up donations per week, dropping the weeks that have no donations
agg_don = donations.groupby('user').apply(
    lambda df: df.amount.resample("W-MON").sum().dropna())